In [99]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib as mpl
import numpy as np
import scipy 
from math import sqrt 
import random

#estimators
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR 
from sklearn import linear_model

#model metrics 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score 

#cross validation 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

#Classification models 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#Evaluating the Results
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix


In [4]:
#data
rawData = pd.read_csv('C:/Users/heath/Documents/Data Class/Data Science with Python/Task 1 Get Started with Data Science and Python/default_of_credit_card_clients.csv', header =1)
rawData.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [76]:
#preprocessing 

#exclude negative credit limits
processed_data = rawData[rawData.LIMIT_BAL > 0]

#transform predictor variable 
processed_data['default payment next month'] = processed_data['default payment next month'].astype(int)

#bin limit balances
bins = [0, 50000, 140000,  240000, 1000000] #divided by quartiles
##group_names = ['low', 'mid-low', 'mid-high', 'high' ]
group_names = [0, 1, 2, 3 ]
processed_data['LIMIT_BINS'] = pd.cut(processed_data['LIMIT_BAL'], bins, labels=group_names)

#bin limit balances
bins_2 = [0, 20, 30,  40, 50, 60, 100] #divided by quartiles
##group_names = ['low', 'mid-low', 'mid-high', 'high' ]
group_names_2 = [0, 1, 2, 3, 4, 5]
processed_data['AGE'] = pd.cut(processed_data['AGE'], bins_2, labels=group_names_2)

#create active 
processed_data['ACTIVE_IND'] = processed_data['BILL_AMT1'] + processed_data['BILL_AMT2']+ processed_data['BILL_AMT3'] + processed_data['BILL_AMT4'] + processed_data['PAY_AMT1']+ processed_data['PAY_AMT2']+ processed_data['PAY_AMT3']+ processed_data['PAY_AMT4']
processed_data = processed_data[processed_data.ACTIVE_IND != 0]

#sample data
processed_data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,LIMIT_BINS,ACTIVE_IND
0,1,20000,2,2,1,1,2,2,-1,-1,...,0,0,689,0,0,0,0,1,0,8393
1,2,120000,2,2,2,1,-1,2,0,0,...,3261,0,1000,1000,1000,0,2000,1,1,13361
2,3,90000,2,2,2,2,0,0,0,0,...,15549,1518,1500,1000,1000,1000,5000,0,1,76174
3,4,50000,2,2,1,2,0,0,0,0,...,29547,2000,2019,1200,1100,1069,1000,0,0,179147
4,5,50000,1,2,1,4,-1,0,-1,0,...,19131,2000,36681,10000,9000,689,679,0,0,128743


In [90]:
#Feature Selection
Feature_Selected_Data = processed_data.iloc[:,[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25]]
print('Summary of feature sample')
Feature_Selected_Data.head()

Summary of feature sample


,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,LIMIT_BINS
0,2,2,1,1,2,2,-1,-1,-2,-2,...,0,0,0,0,689,0,0,0,0,0
1,2,2,2,1,-1,2,0,0,0,2,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,2,2,2,2,0,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,1
3,2,2,1,2,0,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,1,2,1,4,-1,0,-1,0,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [91]:
#Sampling Data 
#Sampled_Data = Feature_Selected_Data.sample(n=10000,replace="False")
#print('The number of observations in the feature testing set is:',str(Sampled_Data))
#Sampled_Data.head()
print('No down sampling for now')

No down sampling for now


In [92]:
#dependent variable
depVar = processed_data['default payment next month']

In [93]:
depVar.head()

0    1
1    1
2    0
3    0
4    0
Name: default payment next month, dtype: int32

In [94]:
X_train, X_test, y_train, y_test = train_test_split(Feature_Selected_Data, depVar,  test_size=.3, random_state = 0)

In [95]:
X_train.shape, X_test.shape

((20327, 23), (8712, 23))

In [96]:
y_train.shape, y_test.shape

((20327,), (8712,))

In [97]:
#Model Fitting Random Forest
DTmodel = DecisionTreeClassifier(random_state=5)
DTmodel.fit(X_train,y_train)
##print(cross_val_score(DTmodel, X_train, y_train))
##DTmodel.score(X_train,y_train)

# predictions
DT_predict = DTmodel.predict(X_test)

#print(cross_val_score(DTmodel, X_test, y_test, cv=11))
print(classification_report(y_test, DT_predict))

              precision    recall  f1-score   support

           0       0.83      0.82      0.83      6772
           1       0.41      0.43      0.42      1940

    accuracy                           0.73      8712
   macro avg       0.62      0.63      0.62      8712
weighted avg       0.74      0.73      0.74      8712



In [98]:
#Model Fitting Nearest Neighbors
RFmodel = RandomForestClassifier()
RFmodel.fit(X_train,y_train)
##print(cross_val_score(DTmodel, X_train, y_train))
##DTmodel.score(X_train,y_train)

# predictionsprocessed_data
RF_predict = RFmodel.predict(X_test)

print(classification_report(y_test, RF_predict))
##print("Accuracy:",metrics.accuracy_score(y_test, RF_predict))

C:\Users\heath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.83      0.95      0.88      6772
           1       0.63      0.32      0.42      1940

    accuracy                           0.81      8712
   macro avg       0.73      0.63      0.65      8712
weighted avg       0.79      0.81      0.78      8712



In [100]:
#Model Fitting Nearest Neighbors
KNmodel = KNeighborsClassifier()
KNmodel.fit(X_train,y_train)
##print(cross_val_score(DTmodel, X_train, y_train))
##DTmodel.score(X_train,y_train)

# predictionsprocessed_data
KN_predict = KNmodel.predict(X_test)

print(classification_report(y_test, KN_predict))
##print("Accuracy:",metrics.accuracy_score(y_test, RF_predict))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      6772
           1       0.40      0.16      0.23      1940

    accuracy                           0.76      8712
   macro avg       0.60      0.55      0.55      8712
weighted avg       0.71      0.76      0.72      8712

